# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df = pd.read_csv('/Users/julianuss/Documents/GitHub/lab-neural-networks/your-code/tic-tac-toe.csv')
df.head()

ModuleNotFoundError: No module named 'distutils'

In [6]:
# Looking into the basic details of the DataFrame
print(df.info())
print(df.shape)
print(df.describe())
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
(958, 10)
         TL   TM   TR   ML   MM   MR   BL   BM   BR class
count   958  958  958  958  958  958  958  958  958   958
unique    3    3    3    3    3    3    3    3    3     2
top       x    x    x    x    x    x    x    x    x  True
freq    418  378  418  378  458  378  418  378  418   626
Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')


In [3]:
# One-hot-encoding
df = pd.get_dummies(df, drop_first=True)
df.head()


,class,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,True,False,True,False,True,False,True,False,True,True,False,True,False,False,True,True,False,True,False
1,True,False,True,False,True,False,True,False,True,True,False,True,False,True,False,False,True,True,False
2,True,False,True,False,True,False,True,False,True,True,False,True,False,True,False,True,False,False,True
3,True,False,True,False,True,False,True,False,True,True,False,True,False,True,False,False,False,False,False
4,True,False,True,False,True,False,True,False,True,True,False,True,False,False,False,True,False,False,False


In [4]:
# Seperating input and output
X = df.iloc[:, :-1]  
y = df.iloc[:, -1]

In [7]:
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Convert back to a DataFrame if needed
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns)

# Display the normalized input data and the target data
print("Normalized Input Data:")
print(X_normalized_df.head())

print("\nOutput:")
print(y.head())

Normalized Input Data:
      class      TL_o      TL_x      TM_o      TM_x      TR_o      TR_x  \
0  0.728252 -0.733294  1.136603 -0.724898  1.238706 -0.733294  1.136603   
1  0.728252 -0.733294  1.136603 -0.724898  1.238706 -0.733294  1.136603   
2  0.728252 -0.733294  1.136603 -0.724898  1.238706 -0.733294  1.136603   
3  0.728252 -0.733294  1.136603 -0.724898  1.238706 -0.733294  1.136603   
4  0.728252 -0.733294  1.136603 -0.724898  1.238706 -0.733294  1.136603   

       ML_o      ML_x      MM_o      MM_x      MR_o      MR_x      BL_o  \
0 -0.724898  1.238706  1.348201 -0.957079  1.379504 -0.807294 -0.733294   
1 -0.724898  1.238706  1.348201 -0.957079  1.379504 -0.807294  1.363709   
2 -0.724898  1.238706  1.348201 -0.957079  1.379504 -0.807294  1.363709   
3 -0.724898  1.238706  1.348201 -0.957079  1.379504 -0.807294  1.363709   
4 -0.724898  1.238706  1.348201 -0.957079  1.379504 -0.807294 -0.733294   

       BL_x      BM_o      BM_x      BR_o  
0  1.136603  1.379504 -0.807294

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here